In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import os.path as path
import copy

from gait_analysis import AnnotationsCasia as Annotations
from gait_analysis import CasiaDataset
from gait_analysis.Config import Config
from gait_analysis import Composer


# Settings 
## Dataset

In [2]:
crop_im_size = [186,250]
c = Config()
c.config['indexing']['grouping'] = 'person_sequence_angle'
c.config['pose']['load'] = True
c.config['flow']['load'] = True
c.config['heatmaps']['load'] = True
c.config['dataset_output'] = {
        'data': ["scenes","flows","heatmaps_LAnkle","heatmaps_RAnkle"],
        'label': "annotations"}
composer = Composer()
transformer = composer.compose()
dataset = CasiaDataset(transform=transformer)


loading configuration  default


## neural network

In [3]:
#DESIGN PARAMETERS FOR NEURAL NETWORK
NR_LSTM_UNITS = 2
IMAGE_INPUT_SIZE_W = 640
IMAGE_INPUT_SIZE_H = 480

IMAGE_AFTER_CONV_SIZE_W = 18
IMAGE_AFTER_CONV_SIZE_H = 13
#for 3x3 kernels, n=num_layers: len_in = 2^n*len_out + sum[i=1..n](2^i)
#CONV_LAYER_LENGTH = 5

LSTM_IO_SIZE = 18*13
LSTM_HIDDEN_SIZE = 18*13

RGB_CHANNELS = 3
TIMESTEPS = 10
BATCH_SIZE = 5 #until now just batch_size = 1

#USE RANDOM IMAGES TO SET UP WORKING EXAMPLE
class TEST_CNN_LSTM(nn.Module):
    def __init__(self):
        super(TEST_CNN_LSTM, self).__init__()
        self.conv1 = nn.Conv2d(3,6,3) #input 640x480
        self.pool1 = nn.MaxPool2d(2,2) #input 638x478 output 319x239
        self.conv2 = nn.Conv2d(6,16,3) # input 319x239 output 317x237
        self.pool2 = nn.MaxPool2d(2,2) # input 317x237 output 158x118
        self.conv3 = nn.Conv2d(16,6,3) # input 158x118 output 156x116
        self.pool3 = nn.MaxPool2d(2,2) # input 156x116 output 78x58
        self.conv4 = nn.Conv2d(6,3,3)  # input 78x58 output 76x56
        self.pool4 = nn.MaxPool2d(2,2) # input 76x56 output 39x29
        self.conv5 = nn.Conv2d(3,1,3)  # input 39x29 output 37x27
        self.pool5 = nn.MaxPool2d(2,2) #output 37x27 output 18x13
        self.lstm = nn.LSTM(LSTM_IO_SIZE,
                            LSTM_HIDDEN_SIZE,
                            NR_LSTM_UNITS)
        self.fc1 = nn.Linear(LSTM_IO_SIZE,120)
        self.fc2 = nn.Linear(120,20)
        self.fc3 = nn.Linear(20,3)
        
        #initialize hidden states of LSTM
        self.hidden = self.init_hidden()
        #print("Hidden:", _hidden)
    def init_hidden(self):
        return (torch.randn(NR_LSTM_UNITS, BATCH_SIZE, LSTM_HIDDEN_SIZE), 
                torch.randn(NR_LSTM_UNITS, BATCH_SIZE, LSTM_HIDDEN_SIZE))
    def forward(self,x):
        # print("Input:", x.size())
        x_arr = torch.zeros(TIMESTEPS,BATCH_SIZE,1,IMAGE_AFTER_CONV_SIZE_H,IMAGE_AFTER_CONV_SIZE_W)
        ## print("X arr size", x_arr.size())
        for i in range(TIMESTEPS):#parallel convolutions which are later concatenated for LSTM
            x_tmp_c1 = self.pool1(F.relu(self.conv1(x[i].float())))
            x_tmp_c2 = self.pool2(F.relu(self.conv2(x_tmp_c1)))
            x_tmp_c3 = self.pool3(F.relu(self.conv3(x_tmp_c2)))
            x_tmp_c4 = self.pool4(F.relu(self.conv4(x_tmp_c3)))
            x_tmp_c5 = self.pool5(F.relu(self.conv5(x_tmp_c4)))
            x_arr[i] = x_tmp_c5 # torch.squeeze(x_tmp_c5)
        # x_tmp_c1 = self.pool1(F.relu(self.conv1(x)))
        # x_tmp_c2 = self.pool2(F.relu(self.conv2(x_tmp_c1)))
        # x_tmp_c3 = self.pool3(F.relu(self.conv3(x_tmp_c2)))
        # x_tmp_c4 = self.pool4(F.relu(self.conv4(x_tmp_c3)))
        # x_tmp_c5 = self.pool5(F.relu(self.conv5(x_tmp_c4)))
        # x_arr = x_tmp_c5 # torch.squeeze(x_tmp_c5)
        
        x, hidden = self.lstm(x_arr.view(TIMESTEPS,BATCH_SIZE,-1), self.hidden)
        # the reshaping was taken from the documentation... and makes scense
        x = x.view(TIMESTEPS,5,1,LSTM_HIDDEN_SIZE) #output.view(seq_len, batch, num_directions, hidden_size)
        x = torch.squeeze(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = x.permute(1,2,0)
        return x
print("Class defined")

Class defined


## Training actual data

In [5]:
#TRAINING
test_net = TEST_CNN_LSTM()
criterion = nn.CrossEntropyLoss()
#criterion = nn.BCELoss()
optimizer = optim.SGD(test_net.parameters(), lr=0.1, momentum=0.9)

dataloader = DataLoader(dataset, batch_size=5)
n_batches = len(dataloader)

#Time for printing
training_start_time = time.time()
    
print('Start training...')
for epoch in range(100): 
    running_loss = 0.0
    print_every = n_batches // 10
    start_time = time.time()
    total_train_loss = 0
    
    print("Epoch:", epoch)
    running_loss = 0.0
    for i, batch in enumerate(dataloader):
        
        inputs, labels_all = batch
        labels = labels_all[:,0:TIMESTEPS]
        optimizer.zero_grad() 
        outputs = test_net(inputs['scenes'])
        # print("Out:", len(outputs), outputs.size())
        # print("Labels_all:", len(labels_all), labels_all.size())
        # print("Labels:", len(labels), labels.size())
        loss = criterion(outputs,labels.long() )
        loss.backward() 
        optimizer.step()
        
        #Print statistics
        # print(loss.data.item())
        running_loss += loss.data.item()
        total_train_loss += loss.data.item()
        
        #Print every 10th batch of an epoch
        if (i + 1) % (print_every + 1) == 0:
            print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                    epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
            #Reset running loss and time
            running_loss = 0.0
            start_time = time.time()

print('...Training finished')

Start training...
Epoch: 0


/home/ron/anaconda3/envs/gait_37/lib/python3.7/site-packages/pandas/core/computation/expressions.py:180: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


Epoch 1, 11% 	 train_loss: 2.21 took: 27.64s
Epoch 1, 22% 	 train_loss: 2.09 took: 28.52s
Epoch 1, 33% 	 train_loss: 2.30 took: 30.36s
Epoch 1, 44% 	 train_loss: 2.84 took: 29.44s
Epoch 1, 55% 	 train_loss: 1.86 took: 23.57s
Epoch 1, 66% 	 train_loss: 2.14 took: 27.28s
Epoch 1, 77% 	 train_loss: 2.33 took: 30.54s
Epoch 1, 88% 	 train_loss: 1.82 took: 25.68s
Epoch 1, 100% 	 train_loss: 2.10 took: 30.69s
Epoch: 1
Epoch 2, 11% 	 train_loss: 2.04 took: 26.39s
Epoch 2, 22% 	 train_loss: 1.53 took: 27.79s
Epoch 2, 33% 	 train_loss: 2.57 took: 29.31s
Epoch 2, 44% 	 train_loss: 3.23 took: 28.32s
Epoch 2, 55% 	 train_loss: 1.81 took: 22.47s
Epoch 2, 66% 	 train_loss: 2.09 took: 25.98s
Epoch 2, 77% 	 train_loss: 2.24 took: 29.34s
Epoch 2, 88% 	 train_loss: 1.89 took: 24.59s
Epoch 2, 100% 	 train_loss: 2.10 took: 29.63s
Epoch: 2
Epoch 3, 11% 	 train_loss: 1.97 took: 25.36s
Epoch 3, 22% 	 train_loss: 1.50 took: 26.97s
Epoch 3, 33% 	 train_loss: 2.62 took: 29.35s
Epoch 3, 44% 	 train_loss: 3.32 too

Epoch 20, 66% 	 train_loss: 1.80 took: 25.56s
Epoch 20, 77% 	 train_loss: 2.10 took: 28.69s
Epoch 20, 88% 	 train_loss: 1.75 took: 24.00s
Epoch 20, 100% 	 train_loss: 2.28 took: 29.13s
Epoch: 20
Epoch 21, 11% 	 train_loss: 1.50 took: 24.96s
Epoch 21, 22% 	 train_loss: 1.28 took: 27.15s
Epoch 21, 33% 	 train_loss: 2.82 took: 28.96s
Epoch 21, 44% 	 train_loss: 3.59 took: 28.13s
Epoch 21, 55% 	 train_loss: 1.79 took: 22.42s
Epoch 21, 66% 	 train_loss: 1.83 took: 25.27s
Epoch 21, 77% 	 train_loss: 2.04 took: 29.00s
Epoch 21, 88% 	 train_loss: 1.85 took: 23.83s
Epoch 21, 100% 	 train_loss: 2.30 took: 29.47s
Epoch: 21
Epoch 22, 11% 	 train_loss: 1.60 took: 25.13s
Epoch 22, 22% 	 train_loss: 1.39 took: 26.93s
Epoch 22, 33% 	 train_loss: 2.73 took: 29.21s
Epoch 22, 44% 	 train_loss: 3.46 took: 28.02s
Epoch 22, 55% 	 train_loss: 1.76 took: 22.34s
Epoch 22, 66% 	 train_loss: 1.82 took: 25.37s
Epoch 22, 77% 	 train_loss: 2.05 took: 28.92s
Epoch 22, 88% 	 train_loss: 1.84 took: 24.05s
Epoch 22, 10

Epoch 39, 100% 	 train_loss: 2.35 took: 29.30s
Epoch: 39
Epoch 40, 11% 	 train_loss: 1.51 took: 25.21s
Epoch 40, 22% 	 train_loss: 1.41 took: 27.07s
Epoch 40, 33% 	 train_loss: 2.58 took: 29.04s
Epoch 40, 44% 	 train_loss: 3.41 took: 28.05s
Epoch 40, 55% 	 train_loss: 1.81 took: 22.03s
Epoch 40, 66% 	 train_loss: 1.75 took: 26.40s
Epoch 40, 77% 	 train_loss: 2.05 took: 29.24s
Epoch 40, 88% 	 train_loss: 1.67 took: 24.07s
Epoch 40, 100% 	 train_loss: 2.37 took: 29.22s
Epoch: 40
Epoch 41, 11% 	 train_loss: 1.50 took: 25.15s
Epoch 41, 22% 	 train_loss: 1.36 took: 27.24s
Epoch 41, 33% 	 train_loss: 2.62 took: 29.15s
Epoch 41, 44% 	 train_loss: 3.46 took: 28.04s
Epoch 41, 55% 	 train_loss: 1.79 took: 22.01s
Epoch 41, 66% 	 train_loss: 1.78 took: 25.88s
Epoch 41, 77% 	 train_loss: 2.07 took: 28.83s
Epoch 41, 88% 	 train_loss: 1.65 took: 23.91s
Epoch 41, 100% 	 train_loss: 2.34 took: 29.15s
Epoch: 41
Epoch 42, 11% 	 train_loss: 1.51 took: 25.20s
Epoch 42, 22% 	 train_loss: 1.42 took: 26.96s
E

Epoch 59, 33% 	 train_loss: 2.56 took: 29.02s
Epoch 59, 44% 	 train_loss: 3.39 took: 28.09s
Epoch 59, 55% 	 train_loss: 1.80 took: 21.97s
Epoch 59, 66% 	 train_loss: 1.74 took: 25.32s
Epoch 59, 77% 	 train_loss: 2.05 took: 28.74s
Epoch 59, 88% 	 train_loss: 1.63 took: 23.79s
Epoch 59, 100% 	 train_loss: 2.35 took: 29.04s
Epoch: 59
Epoch 60, 11% 	 train_loss: 1.47 took: 24.48s
Epoch 60, 22% 	 train_loss: 1.41 took: 26.56s
Epoch 60, 33% 	 train_loss: 2.57 took: 28.80s
Epoch 60, 44% 	 train_loss: 3.40 took: 28.14s
Epoch 60, 55% 	 train_loss: 1.80 took: 22.03s
Epoch 60, 66% 	 train_loss: 1.73 took: 25.88s
Epoch 60, 77% 	 train_loss: 2.05 took: 28.73s
Epoch 60, 88% 	 train_loss: 1.64 took: 23.88s
Epoch 60, 100% 	 train_loss: 2.35 took: 28.83s
Epoch: 60
Epoch 61, 11% 	 train_loss: 1.47 took: 24.95s
Epoch 61, 22% 	 train_loss: 1.41 took: 26.93s
Epoch 61, 33% 	 train_loss: 2.57 took: 29.12s
Epoch 61, 44% 	 train_loss: 3.40 took: 28.24s
Epoch 61, 55% 	 train_loss: 1.80 took: 22.07s
Epoch 61, 66

Epoch 78, 66% 	 train_loss: 1.70 took: 25.69s
Epoch 78, 77% 	 train_loss: 2.03 took: 29.23s
Epoch 78, 88% 	 train_loss: 1.62 took: 24.15s
Epoch 78, 100% 	 train_loss: 2.34 took: 29.40s
Epoch: 78
Epoch 79, 11% 	 train_loss: 1.45 took: 24.95s
Epoch 79, 22% 	 train_loss: 1.40 took: 27.10s
Epoch 79, 33% 	 train_loss: 2.54 took: 29.69s
Epoch 79, 44% 	 train_loss: 3.38 took: 28.00s
Epoch 79, 55% 	 train_loss: 1.80 took: 22.55s
Epoch 79, 66% 	 train_loss: 1.70 took: 25.39s
Epoch 79, 77% 	 train_loss: 2.03 took: 28.97s
Epoch 79, 88% 	 train_loss: 1.63 took: 23.95s
Epoch 79, 100% 	 train_loss: 2.35 took: 28.79s
Epoch: 79
Epoch 80, 11% 	 train_loss: 1.43 took: 24.65s
Epoch 80, 22% 	 train_loss: 1.41 took: 27.19s
Epoch 80, 33% 	 train_loss: 2.54 took: 29.35s
Epoch 80, 44% 	 train_loss: 3.39 took: 28.05s
Epoch 80, 55% 	 train_loss: 1.80 took: 22.31s
Epoch 80, 66% 	 train_loss: 1.71 took: 25.60s
Epoch 80, 77% 	 train_loss: 2.04 took: 29.26s
Epoch 80, 88% 	 train_loss: 1.61 took: 24.16s
Epoch 80, 10

Epoch 97, 100% 	 train_loss: 2.35 took: 29.24s
Epoch: 97
Epoch 98, 11% 	 train_loss: 1.42 took: 24.97s
Epoch 98, 22% 	 train_loss: 1.45 took: 26.85s
Epoch 98, 33% 	 train_loss: 2.47 took: 29.25s
Epoch 98, 44% 	 train_loss: 3.33 took: 27.79s
Epoch 98, 55% 	 train_loss: 1.81 took: 22.14s
Epoch 98, 66% 	 train_loss: 1.68 took: 25.92s
Epoch 98, 77% 	 train_loss: 2.03 took: 28.62s
Epoch 98, 88% 	 train_loss: 1.57 took: 24.22s
Epoch 98, 100% 	 train_loss: 2.36 took: 29.45s
Epoch: 98
Epoch 99, 11% 	 train_loss: 1.41 took: 25.09s
Epoch 99, 22% 	 train_loss: 1.45 took: 26.80s
Epoch 99, 33% 	 train_loss: 2.47 took: 28.80s
Epoch 99, 44% 	 train_loss: 3.34 took: 27.98s
Epoch 99, 55% 	 train_loss: 1.81 took: 22.42s
Epoch 99, 66% 	 train_loss: 1.67 took: 25.52s
Epoch 99, 77% 	 train_loss: 2.02 took: 28.65s
Epoch 99, 88% 	 train_loss: 1.58 took: 23.95s
Epoch 99, 100% 	 train_loss: 2.36 took: 28.89s
Epoch: 99
Epoch 100, 11% 	 train_loss: 1.42 took: 24.33s
Epoch 100, 22% 	 train_loss: 1.43 took: 26.60s